In [ ]:
# df_all.isna().sum()[df_all.isna().sum() > 0]
# seat_remark numeric,
# product_remark numeric,
# transaction_type numeric,
# list_price numeric,
# sale_coupon_id numeric,
# bold_coupon_ids numeric,

SyntaxError: invalid syntax (2208957560.py, line 1)

In [103]:
import pandas as pd
import numpy as np
import json

# 1. 전체 DataFrame에서 문제가 되는 값들 확인
def diagnose_dataframe_issues(df):
    """DataFrame에서 JSON 변환 시 문제가 되는 값들을 진단"""
    
    print("=== DataFrame 진단 결과 ===")
    
    # 1) 무한대 값 확인
    inf_mask = np.isinf(df.select_dtypes(include=[np.number]))
    if inf_mask.any().any():
        print("\n🚨 무한대(inf) 값이 발견된 컬럼들:")
        for col in inf_mask.columns:
            if inf_mask[col].any():
                inf_count = inf_mask[col].sum()
                inf_indices = df[inf_mask[col]].index.tolist()
                print(f"   - {col}: {inf_count}개 (행 인덱스: {inf_indices[:10]}{'...' if len(inf_indices) > 10 else ''})")
    
    # 2) NaN 값 확인
    nan_mask = df.isnull()
    if nan_mask.any().any():
        print("\n⚠️  NaN 값이 발견된 컬럼들:")
        for col in nan_mask.columns:
            if nan_mask[col].any():
                nan_count = nan_mask[col].sum()
                print(f"   - {col}: {nan_count}개")
    
    # 3) 매우 큰 숫자 확인 (JSON 범위 초과 가능성)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(f"\n📊 숫자형 컬럼들의 최대/최소값:")
    for col in numeric_cols:
        if not df[col].empty:
            max_val = df[col].max()
            min_val = df[col].min()
            print(f"   - {col}: min={min_val}, max={max_val}")
            
            # 매우 큰 값 경고
            if abs(max_val) > 1e308 or abs(min_val) > 1e308:
                print(f"     ⚠️  {col}에 매우 큰 값이 있어 JSON 변환 시 문제가 될 수 있습니다")

# 2. 구체적으로 어떤 행이 문제인지 찾기
def find_problematic_rows(df):
    """JSON 변환이 불가능한 행들을 찾아서 반환"""
    
    problematic_rows = []
    
    for idx, row in df.iterrows():
        try:
            # 각 행을 JSON으로 변환 시도
            json.dumps(row.to_dict())
        except (ValueError, TypeError) as e:
            problematic_rows.append({
                'index': idx,
                'error': str(e),
                'row_data': row.to_dict()
            })
            
            # 처음 10개만 출력
            if len(problematic_rows) <= 10:
                print(f"❌ 행 {idx}: {e}")
    
    return problematic_rows

# 3. 데이터 정리 함수
def clean_dataframe_for_json(df):
    """DataFrame을 JSON 호환 가능하도록 정리"""
    
    df_cleaned = df.copy()
    
    # 1) 무한대 값을 None으로 변경
    df_cleaned = df_cleaned.replace([np.inf, -np.inf], None)
    
    # 2) NaN을 None으로 변경
    df_cleaned = df_cleaned.where(pd.notnull(df_cleaned), None)
    
    # 3) 매우 큰 숫자들을 적절한 범위로 제한 (선택사항)
    numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        # JavaScript Number.MAX_SAFE_INTEGER 범위로 제한
        mask = df_cleaned[col].abs() > 9007199254740991
        if mask.any():
            print(f"⚠️  {col} 컬럼의 매우 큰 값들을 제한합니다")
            df_cleaned.loc[mask, col] = None
    
    return df_cleaned

# 사용 예시
if __name__ == "__main__":
    # DataFrame 진단
    print("1단계: DataFrame 문제 진단")
    diagnose_dataframe_issues(df_all)
    
    print("\n" + "="*50)
    print("2단계: 문제가 되는 행들 찾기")
    problematic_rows = find_problematic_rows(df_all)
    
    if problematic_rows:
        print(f"\n총 {len(problematic_rows)}개의 문제 행이 발견되었습니다.")
    else:
        print("\n✅ 모든 행이 JSON 변환 가능합니다.")
    
    print("\n" + "="*50)
    print("3단계: 데이터 정리 및 업로드")
    
    # 데이터 정리
    df_cleaned = clean_dataframe_for_json(df_all)
    
    try:
        # 정리된 데이터로 Supabase 업로드
        data_to_insert = df_cleaned.to_dict('records')
        result = supabase.table('ticketbay_data').insert(data_to_insert).execute()
        print(f"✅ Supabase 업로드 성공: {len(data_to_insert)}건")
        
    except Exception as e:
        print(f"❌ 여전히 업로드 실패: {e}")
        
        # 더 세밀한 디버깅을 위해 작은 배치로 테스트
        print("\n🔍 작은 배치로 테스트 중...")
        batch_size = 10
        for i in range(0, min(100, len(data_to_insert)), batch_size):
            batch = data_to_insert[i:i+batch_size]
            try:
                supabase.table('ticketbay_data').insert(batch).execute()
                print(f"✅ 배치 {i//batch_size + 1} 성공")
            except Exception as batch_error:
                print(f"❌ 배치 {i//batch_size + 1} 실패: {batch_error}")
                print(f"   문제 데이터: {batch}")
                break

1단계: DataFrame 문제 진단
=== DataFrame 진단 결과 ===

⚠️  NaN 값이 발견된 컬럼들:
   - end_perform_date: 73개
   - floor: 12개
   - addinfo: 2847개
   - seat_remark: 7039개
   - product_remark: 7039개
   - seat_remark_str: 6588개
   - product_remark_str: 6381개
   - transaction_type: 7039개
   - ticket_receive_date: 3164개
   - is_adult: 6242개
   - list_price: 12개
   - description: 832개
   - sale_coupon_id: 6762개
   - deal_detail_loc: 7038개
   - deal_prefer_loc: 7036개
   - bold_coupon_ids: 6998개

📊 숫자형 컬럼들의 최대/최소값:
   - id: min=6581635, max=7674621
   - member_id: min=3760, max=3131801
   - category_id: min=195, max=5932
   - display_number: min=30259943532, max=55672145837662
   - depth1_id: min=3, max=3
   - depth2_id: min=195, max=5932
   - depth3_id: min=30, max=5934
   - sale_quantity: min=1, max=8
   - price: min=10000.0, max=3800000.0
   - total_price: min=10000.0, max=8000000.0
   - seat_remark: min=nan, max=nan
   - product_remark: min=nan, max=nan
   - transaction_type: min=nan, max=nan
   - list_pri

In [ ]:
import pandas as pd
import numpy as np
import json

def prepare_for_supabase(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # NaN, inf → None
    df = df.replace({np.nan: None, np.inf: None, -np.inf: None})

    # 날짜 형식 문자열로 변환 (datetime 컬럼은 ISO 포맷으로)
    date_cols = ['start_perform_date', 'end_perform_date', 'created_at']
    just_date_cols = ['perform_date', 'ticket_receive_date']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S') if pd.notna(x) else None)
    for col in just_date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').apply(lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else None)

    # boolean 처리
    bool_cols = [
        'is_together', 'is_use_safe', 'is_have_ticket', 'is_adult',
        'is_list_price', 'is_use_pin', 'is_use_delivery', 'is_use_field',
        'is_use_immediately', 'is_use_etc', 'is_agree_global', 'is_exist_compare'
    ]
    for col in bool_cols:
        if col in df.columns:
            df[col] = df[col].map({'YES': True, 'NO': False, True: True, False: False})

    # text 강제 처리 (숫자이지만 Supabase에 text로 저장되는 컬럼들)
    text_cast_cols = [
        'seat_remark', 'product_remark', 'transaction_type',
        'list_price', 'sale_coupon_id', 'bold_coupon_ids'
    ]
    for col in text_cast_cols:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: str(x) if x is not None else None)

    return df


=== 데이터 정리 시작 ===
1. 원본 데이터 문제 확인:
=== JSON 호환성 검증 ===
⚠️  end_perform_date: NaN=73, Inf=0
⚠️  floor: NaN=12, Inf=0
⚠️  addinfo: NaN=2847, Inf=0
⚠️  seat_remark: NaN=7039, Inf=0
⚠️  product_remark: NaN=7039, Inf=0
⚠️  seat_remark_str: NaN=6588, Inf=0
⚠️  product_remark_str: NaN=6381, Inf=0
⚠️  transaction_type: NaN=7039, Inf=0
⚠️  ticket_receive_date: NaN=3164, Inf=0
⚠️  is_adult: NaN=6242, Inf=0
⚠️  list_price: NaN=12, Inf=0
⚠️  description: NaN=832, Inf=0
⚠️  sale_coupon_id: NaN=6762, Inf=0
⚠️  deal_detail_loc: NaN=7038, Inf=0
⚠️  deal_prefer_loc: NaN=7036, Inf=0
⚠️  bold_coupon_ids: NaN=6998, Inf=0

=== 샘플 행 JSON 변환 테스트 ===
✅ 행 0: JSON 변환 성공
✅ 행 1: JSON 변환 성공
✅ 행 2: JSON 변환 성공
✅ 행 3: JSON 변환 성공
✅ 행 4: JSON 변환 성공

2. 데이터 정리 중...

3. 정리된 데이터 검증:
=== JSON 호환성 검증 ===
⚠️  end_perform_date: NaN=73, Inf=0
⚠️  floor: NaN=12, Inf=0
⚠️  addinfo: NaN=2847, Inf=0
⚠️  seat_remark: NaN=7039, Inf=0
⚠️  product_remark: NaN=7039, Inf=0
⚠️  seat_remark_str: NaN=6588, Inf=0
⚠️  product_remark_str: NaN

In [93]:
import pandas as pd

df_all = pd.read_csv('ticketbay_log/ticketbay_2025-07-25_20-07-39.csv')

df_all.head(1)


,id,member_id,category_id,display_number,name,perform_date_type,start_perform_date,end_perform_date,perform_date,depth1_id,...,description,sale_coupon_id,created_at,ip_addr,deal_detail_loc,deal_prefer_loc,bold_coupon_ids,is_agree_global,is_exist_compare,category_name
0,7673354,2824513,5123,2633250742618,W43구역(존/블럭) 5열,FIXED,2025-07-26T20:00:00,2025-07-26T20:00:00,2025-07-26T20:00,3,...,19세 공연-신분증 보유 필요\n(구매자 신분증으로 성인인증 가능)\n현장거래\n구...,NaN,2025-07-25T17:23:19,211.234.181.72,NaN,NaN,NaN,NO,NO,칸예 웨스트


In [94]:
for col in df_all.columns:
    dtype = df_all[col].dtype
    if 'int' in str(dtype):
        coltype = 'bigint'
    elif 'float' in str(dtype):
        coltype = 'numeric'
    elif 'datetime' in str(dtype):
        coltype = 'timestamp'
    elif 'bool' in str(dtype):
        coltype = 'boolean'
    else:
        coltype = 'text'
    print(f"{col} {coltype},")

id bigint,
member_id bigint,
category_id bigint,
display_number bigint,
name text,
perform_date_type text,
start_perform_date text,
end_perform_date text,
perform_date text,
depth1_id bigint,
depth1_name text,
depth2_id bigint,
depth2_name text,
depth3_id bigint,
depth3_name text,
info_type text,
seat_number_type text,
seat_number text,
area text,
floor text,
grade text,
addinfo text,
sale_quantity bigint,
is_together text,
price numeric,
total_price numeric,
seat_remark numeric,
product_remark numeric,
seat_remark_str text,
product_remark_str text,
transaction_type numeric,
product_status text,
is_use_safe text,
is_have_ticket text,
ticket_receive_date text,
is_adult text,
list_price numeric,
is_list_price text,
is_use_pin text,
is_use_delivery text,
is_use_field text,
is_use_immediately text,
is_use_etc text,
description text,
sale_coupon_id numeric,
created_at text,
ip_addr text,
deal_detail_loc text,
deal_prefer_loc text,
bold_coupon_ids numeric,
is_agree_global text,
is_exist_comp

In [95]:
# 숫자 컬럼에 문자열 들어갔는지 확인
for col in df_all.select_dtypes(include='object').columns:
    print(f"{col}: {df_all[col].unique()[:5]}")

name: ['W43구역(존/블럭) 5열 ' 'N12구역(존/블럭) 9ROW ' 'W10구역(존/블럭) 3열ROW '
 'E53구역(존/블럭) 14ROW 11,12번' 'W34구역(존/블럭) 1ROW 4연석 -아옮-']
perform_date_type: ['FIXED']
start_perform_date: ['2025-07-26T20:00:00' '2025-08-02T17:00:00' '2025-08-01T19:30:00'
 '2025-09-27T19:00:00' '2025-08-23T18:00:00']
end_perform_date: ['2025-07-26T20:00:00' '2025-08-02T17:00:00' '2025-08-01T19:30:00'
 '2025-09-27T19:00:00' '2025-08-23T18:00:00']
perform_date: ['2025-07-26T20:00' '2025-08-02T17:00' '2025-08-01T19:30'
 '2025-09-27T19:00' '2025-08-23T18:00']
depth1_name: ['콘서트']
depth2_name: ['칸예 웨스트' '박보검' '뮤즈 (MUSE)' '호시x우지' '악뮤 (AKMU)']
depth3_name: ['칸예 웨스트 2025 - 인천' '박보검 2025 - 서울' '뮤즈 2025 - 인천' '호시x우지 2025 - 광주'
 '악뮤 2025 - 서울']
info_type: ['CONCERT_B' 'CONCERT_A']
seat_number_type: ['ROW' 'ENTER']
seat_number: ['5' '9' '3열' '14' '1']
area: ['W43' 'N12' 'W10' 'E53' 'W34']
floor: ['4층 북 (4F NORTH)' '2층 북 (1F NORTH)' '2층 서 (2F WEST)' '4층 동 (4F EAST)'
 '4층 서 (4F WEST)']
grade: ['지정석 S' '지정석 P' '지정석 A' '지정석 R' 'S']
ad

In [ ]:
import pandas as pd
import numpy as np
import json

def prepare_for_supabase(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # NaN, inf → None
    df = df.replace({np.nan: None, np.inf: None, -np.inf: None})

    # 날짜 형식 문자열로 변환 (datetime 컬럼은 ISO 포맷으로)
    date_cols = ['start_perform_date', 'end_perform_date', 'created_at']
    just_date_cols = ['perform_date', 'ticket_receive_date']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S') if pd.notna(x) else None)
    for col in just_date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').apply(lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else None)

    # boolean 처리
    bool_cols = [
        'is_together', 'is_use_safe', 'is_have_ticket', 'is_adult',
        'is_list_price', 'is_use_pin', 'is_use_delivery', 'is_use_field',
        'is_use_immediately', 'is_use_etc', 'is_agree_global', 'is_exist_compare'
    ]
    for col in bool_cols:
        if col in df.columns:
            df[col] = df[col].map({'YES': True, 'NO': False, True: True, False: False})

    # text 강제 처리 (숫자이지만 Supabase에 text로 저장되는 컬럼들)
    text_cast_cols = [
        'seat_remark', 'product_remark', 'transaction_type',
        'list_price', 'sale_coupon_id', 'bold_coupon_ids'
    ]
    for col in text_cast_cols:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: str(x) if x is not None else None)

    return df


ValueError: Out of range float values are not JSON compliant

In [84]:
bad_vals = df_clean[
    df_clean.applymap(lambda x: pd.isna(x) or x in [float("inf"), float("-inf")])
]

print("🧨 문제 있는 행 수:", len(bad_vals))
print(bad_vals.head())

🧨 문제 있는 행 수: 7039
   id  member_id  category_id  display_number name perform_date_type  \
0 NaN        NaN          NaN             NaN  NaN               NaN   
1 NaN        NaN          NaN             NaN  NaN               NaN   
2 NaN        NaN          NaN             NaN  NaN               NaN   
3 NaN        NaN          NaN             NaN  NaN               NaN   
4 NaN        NaN          NaN             NaN  NaN               NaN   

  start_perform_date end_perform_date perform_date  depth1_id  ...  \
0                NaN              NaN          NaN        NaN  ...   
1                NaN              NaN          NaN        NaN  ...   
2                NaN              NaN          NaN        NaN  ...   
3                NaN              NaN          NaN        NaN  ...   
4                NaN              NaN          NaN        NaN  ...   

  description  sale_coupon_id created_at  ip_addr deal_detail_loc  \
0         NaN            None        NaN      NaN          

C:\Users\torysaewoo\AppData\Local\Temp\ipykernel_108524\181220420.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean.applymap(lambda x: pd.isna(x) or x in [float("inf"), float("-inf")])


In [62]:
import requests
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import time  # ⏱ 시간 측정용 추가
# ──────────────────────────────
# 1. 카테고리 정보 수집
# ──────────────────────────────
start = time.time()  # ⏱ 시작 시간 기록

url_categories = "https://www.ticketbay.co.kr/ticketbayApi/content/v1/public/categories"
headers_categories = {
    "accept": "application/json",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

res_cat = requests.get(url_categories, headers=headers_categories)
result = res_cat.json()

# 하위 카테고리 (콘서트 종류)
concert_categories = [
    {"id": item["id"], "name": item["name"]}
    for item in result["data"][0]["children"]
]

# ──────────────────────────────
# 2. 요청 함수 정의 (에러 처리 포함)
# ──────────────────────────────
def fetch_products(category):
    url = "https://www.ticketbay.co.kr/ticketbayApi/product/v1/public/products"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
    }

    payload = {
        "category_id": str(category["id"]),
        "page": "0",
        "size": 10000,
        "offset": 0
    }

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
        response.raise_for_status()
        result = response.json()
        items = result.get("data", {}).get("content", [])

        for item in items:
            item["category_id"] = category["id"]
            item["category_name"] = category["name"]

        print(f"✅ {category['name']} ({category['id']}) → {len(items)}개 수집됨")
        return items

    except Exception as e:
        print(f"❌ {category['name']} ({category['id']}) 오류: {e}")
        return []


# ──────────────────────────────
# 3. 병렬 요청 실행 (최대 10개 동시 요청)
# ──────────────────────────────
all_items = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_products, cat) for cat in concert_categories]

    for future in as_completed(futures):
        items = future.result()
        all_items.extend(items)

# ──────────────────────────────
# 4. 데이터 저장
# ──────────────────────────────
# ───────── 저장 + 시간 측정 ─────────
df_all = pd.DataFrame(all_items)
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"ticketbay/ticketbay_{timestamp}.csv"

df_all.to_csv(filename, index=False, encoding="utf-8-sig")

end = time.time()
elapsed = end - start

# ⏱ 소요 시간 출력
print(f"\n🎉 전체 저장 완료: {filename} (총 {len(df_all)}건)")
print(f"⏱ 총 소요 시간: {elapsed:.2f}초")

✅ 칸예 웨스트 (5123) → 8개 수집됨
✅ 박보검 (844) → 6개 수집됨
✅ 뮤즈 (MUSE) (5904) → 194개 수집됨
✅ 호시x우지 (5854) → 117개 수집됨
✅ 악뮤 (AKMU) (840) → 42개 수집됨
✅ 월드 오브 스트릿 우먼 파이터 (5932) → 277개 수집됨
✅ BOYNEXTDOOR (4945) → 276개 수집됨
✅ 강다니엘 (1174) → 9개 수집됨
✅ 결속밴드 (KESSOKUBAND) (5930) → 58개 수집됨
✅ 10cm (216) → 20개 수집됨
✅ 플레이브 (PLAVE) (4883) → 153개 수집됨
✅ 잔나비 (2041) → 50개 수집됨
✅ 권진아 (375) → 0개 수집됨
✅ natori (5863) → 97개 수집됨
✅ 남우현 (3451) → 44개 수집됨
✅ 로버트 글래스퍼 (5920) → 4개 수집됨
✅ 마이 케미컬 로맨스 (My Chemical Romance) (5913) → 36개 수집됨
✅ 더보이즈 (THE BOYZ) (2119) → 657개 수집됨
✅ 슈퍼주니어 (1347) → 347개 수집됨
✅ Minami (5884) → 38개 수집됨
✅ 멜로망스 (1338) → 0개 수집됨
✅ 데이식스 (DAY6) (1916) → 132개 수집됨
✅ 세카이노오와리 (5873) → 131개 수집됨
✅ 미스터트롯 3 (5592) → 128개 수집됨
✅ 실리카겔 (4582) → 117개 수집됨
✅ 어썸스테이지 (3312) → 30개 수집됨
✅ 온유 (4145) → 22개 수집됨
✅ 싸이 (195) → 121개 수집됨
✅ FT아일랜드 (446) → 36개 수집됨
✅ 오아시스 (5374) → 169개 수집됨
✅ 에스파 (3767) → 1772개 수집됨
✅ 유우리 (5160) → 27개 수집됨
✅ 위너 (2438) → 44개 수집됨
✅ WayV (5405) → 99개 수집됨
✅ 케플러 (3873) → 7개 수집됨
✅ 정은지 (1040) → 4개 수집됨
✅ 타일러, 더 크리에이터 (5566) → 115개 수

In [52]:
import requests
import pandas as pd
from datetime import datetime

# 1. 콘서트 카테고리 리스트 뽑기
url_categories = "https://www.ticketbay.co.kr/ticketbayApi/content/v1/public/categories"
headers_categories = {
    "accept": "application/json",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

res_cat = requests.get(url_categories, headers=headers_categories)
result = res_cat.json()

# 콘서트 리스트 (id와 name)
concert_list = [
    {"id": item["id"], "name": item["name"]}
    for item in result["data"][0]["children"]
]
df_concert = pd.DataFrame(concert_list)
category_ids = df_concert["id"].unique().tolist()

# 2. category_id를 바꿔가며 상품 데이터 수집
url_products = "https://www.ticketbay.co.kr/ticketbayApi/product/v1/public/products"
headers_products = {
    "Accept": "application/json",
    "Content-Type": "application/json",
}

all_data = []  # 누적 리스트

for category_id in category_ids:
    data = {
        "category_id": str(category_id),
        "page": "0",
        "size": 10000,
        "offset": 0
    }

    response = requests.post(url_products, headers=headers_products, json=data)
    result2 = response.json()

    # 데이터가 있으면 누적
    if "data" in result2 and "content" in result2["data"]:
        all_data.extend(result2["data"]["content"])
        print(f"✅ category_id={category_id} 완료 (상품 {len(result2['data']['content'])}개)")
    else:
        print(f"⚠ category_id={category_id} 데이터 없음")

# 3. DataFrame으로 변환 후 저장
df_all = pd.DataFrame(all_data)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
df_all.to_csv(f"ticketbay/ticketbay_{timestamp}.csv", index=False, encoding="utf-8-sig")

print(f"🎉 전체 데이터 저장 완료: ticketbay/ticketbay_{timestamp}.csv")


{'code': 'OK', 'message': '요청이 성공하였습니다', 'data': [{'id': 3, 'depth1_id': None, 'level': 1, 'name': '콘서트', 'depth1_name': None, 'depth2_name': None, 'depth3_name': None, 'count': None, 'sort': 1, 'is_bold': None, 'children': [{'id': 1916, 'depth1_id': None, 'level': 2, 'name': '데이식스 (DAY6)', 'depth1_name': None, 'depth2_name': None, 'depth3_name': None, 'count': 130, 'sort': 0, 'is_bold': 'YES', 'children': [{'id': 5871, 'depth1_id': None, 'level': 3, 'name': '데이식스 2025 - 서울', 'depth1_name': None, 'depth2_name': None, 'depth3_name': None, 'count': 130, 'sort': 0, 'is_bold': None, 'children': None}]}, {'id': 2119, 'depth1_id': None, 'level': 2, 'name': '더보이즈 (THE BOYZ)', 'depth1_name': None, 'depth2_name': None, 'depth3_name': None, 'count': 654, 'sort': 1, 'is_bold': 'YES', 'children': [{'id': 5896, 'depth1_id': None, 'level': 3, 'name': '더보이즈 2025 - 서울', 'depth1_name': None, 'depth2_name': None, 'depth3_name': None, 'count': 654, 'sort': 0, 'is_bold': None, 'children': None}]}, {'id': 

In [55]:
concert_list = df['id'].unique().tolist()

In [ ]:
import requests
import pandas as pd
from datetime import datetime

url = "https://www.ticketbay.co.kr/ticketbayApi/product/v1/public/products"
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
}

data = {
    "category_id": "1916",
    "page": "0",
    "size": 10000,
    "offset": 0
}

response = requests.post(url, headers=headers, json=data)
result2 = response.json()

df2 = pd.DataFrame(result2['data']['content'])


timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

df2.to_csv(f'ticketbay/ticketbay_{timestamp}.csv', index=False, encoding='utf-8-sig')

{'code': 'OK', 'message': '요청이 성공하였습니다', 'data': {'content': [{'id': 7672643, 'member_id': 175331, 'category_id': 5871, 'display_number': '9413603101252', 'name': '35구역(존/블럭) 9ROW 사진참고', 'perform_date_type': 'FIXED', 'start_perform_date': '2025-07-26T18:00:00', 'end_perform_date': '2025-07-26T18:00:00', 'perform_date': '2025-07-26T18:00', 'depth1_id': 3, 'depth1_name': '콘서트', 'depth2_id': 1916, 'depth2_name': '데이식스 (DAY6)', 'depth3_id': 5871, 'depth3_name': '데이식스 2025 - 서울', 'info_type': 'CONCERT_B', 'seat_number_type': 'ROW', 'seat_number': '9', 'area': '35', 'floor': '3층 (3F)', 'grade': '지정석', 'addinfo': '사진참고', 'sale_quantity': 1, 'is_together': 'NO', 'price': 290000.0, 'total_price': 290000.0, 'seat_remark': '', 'product_remark': '', 'seat_remark_str': '', 'product_remark_str': '', 'transaction_type': '', 'product_status': 'SALE', 'is_use_safe': 'NO', 'is_have_ticket': 'YES', 'ticket_receive_date': None, 'is_adult': None, 'list_price': 110000.0, 'is_list_price': 'NO', 'is_use_pin':

In [57]:
df2 = pd.DataFrame(result2['data']['content'])
print(df2)

          id  member_id  category_id display_number  \
0    7672643     175331         5871  9413603101252   
1    7673609    2584731         5871  2912645261450   
2    7671939    1756044         5871  9219680093621   
3    7637705    1770700         5871  4012379285781   
4    7673801    2661364         5871  9667400152674   
..       ...        ...          ...            ...   
125  7672841    2075668         5871  3906124971252   
126  7674481    1787932         5871  3913080166248   
127  7650869    1787932         5871  2790962600535   
128  7673527    1787991         5871  4557828523607   
129  7672406    2463695         5871  6301126253361   

                          name perform_date_type   start_perform_date  \
0         35구역(존/블럭) 9ROW 사진참고             FIXED  2025-07-26T18:00:00   
1          34구역(존/블럭) 6ROW 1n번             FIXED  2025-07-26T18:00:00   
2     28구역(존/블럭) 9ROW 본확x 티켓양도             FIXED  2025-07-26T18:00:00   
3       33구역(존/블럭) 13ROW 신분증대여             FIXE

In [61]:
import pandas as pd
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

df2.to_csv(f'ticketbay/ticketbay_{timestamp}.csv', index=False, encoding='utf-8-sig')


In [1]:


import os
import json
import requests
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from openai import OpenAI
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import random
from datetime import datetime

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class InterparkTicketCrawler:
    def __init__(self, creds='google.json', sheet_name='감사한 티켓팅 신청서'):
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(creds, scope)
        self.sheet = gspread.authorize(creds).open(sheet_name).worksheet('Hot')

        # 캐시 파일 경로
        self.artist_cache_path = Path('artist_cache.json')
        self.hashtag_cache_path = Path('hashtag_cache.json')
        self.tweet_cache_path = Path('tweet_cache.json')

        # 캐시 로딩
        self.artist_cache = self.load_cache(self.artist_cache_path)
        self.hashtag_cache = self.load_cache(self.hashtag_cache_path)
        self.tweet_cache = self.load_cache(self.tweet_cache_path)
        
    def load_cache(self, path: Path) -> dict:
        if path.exists():
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                pass
        return {}

    def save_cache(self, cache: dict, path: Path):
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(cache, f, ensure_ascii=False, indent=2)

    def fetch_data(self):
        url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"
        params = {"goodsGenre": "ALL", "goodsRegion": "ALL", "offset": 0, "pageSize": 400, "sorting": "OPEN_ASC"}
        headers = {
            "user-agent": "Mozilla/5.0",
            "referer": "https://tickets.interpark.com/contents/notice"
        }
        r = requests.get(url, params=params, headers=headers)
        r.raise_for_status()
        return r.json()

    def filter_hot(self, data):
        hot = []
        for d in data:
            hot.append({
                '오픈시간': d.get('openDateStr', ''),
                '조회수': d.get('viewCount', 0),
                '예매타입': d.get('openTypeStr', ''),
                '제목': d.get('title', ''),
                '예매코드': d.get('goodsCode', ''),
                '장르': d.get('goodsGenreStr', ''),
                'Image': d.get('posterImageUrl', '')
            })
        return hot


    def run(self):
        raw = self.fetch_data()
        hot = self.filter_hot(raw)
        df = pd.DataFrame(hot)
        if df.empty:
            return df
        df = df.sort_values(by='오픈시간')
        
        return df

df = InterparkTicketCrawler().run()
df.to_csv('all_list.csv', index=False)
if not df.empty:
    print("\n📋 HOT 티켓 요약:")
    print(df[['오픈시간', '조회수','제목',  '장르']].to_string(index=False))


📋 HOT 티켓 요약:
               오픈시간   조회수                                                                              제목      장르
                     4901                                                       2025 데미소다 콘서트, DEMI-CON!      콘서트
2025-07-21 09:00:00    71                                                              리얼공룡쇼 〈포켓다이노〉 - 당진     뮤지컬
2025-07-21 09:00:00  1084                                                                      뮤지컬〈바리케이드〉     뮤지컬
2025-07-21 09:00:00   103                                                             리얼공룡쇼 〈포켓다이노〉 - 대전      뮤지컬
2025-07-21 10:00:00   133                                                                  몽글몽글 꿈공장 - 이천  무용/전통예술
2025-07-21 10:00:00   102                                                    뮤지컬 빨간 내복 야코〈골드버튼을 향해서!〉-대전      뮤지컬
2025-07-21 10:00:00    78                                                         2025 가족매직쇼〈매직키즈마술쇼〉-평택      뮤지컬
2025-07-21 10:00:00   121                                            가족뮤지컬

In [2]:

df['장르'].value_counts()

장르
뮤지컬        20
콘서트        15
연극          5
클래식/오페라     5
무용/전통예술     2
Name: count, dtype: int64

In [3]:
df[df['장르']=='콘서트'].sort_values(by='조회수', ascending=False)

,오픈시간,조회수,예매타입,제목,예매코드,장르,Image
44,2025-07-30 20:00:00,41409,일반예매,TOMORROW X TOGETHER WORLD TOUR 〈ACT : TOMORROW...,25008966,콘서트,https://ticketimage.interpark.com/Play/image/l...
34,2025-07-23 20:00:00,9817,일반예매,2025 실리카겔 단독공연 Syn.THE.Size X,,콘서트,https://ticketimage.interpark.com/TicketImage/...
41,2025-07-25 18:00:00,8169,일반예매,민트페스타 vol.78 SPIRITED,,콘서트,https://ticketimage.interpark.com/TicketImage/...
20,2025-07-22 12:00:00,5214,일반예매,Jacky Cheung 60+ Concert Tour Seoul,25010261,콘서트,https://ticketimage.interpark.com/Play/image/l...
46,,4901,일반예매,"2025 데미소다 콘서트, DEMI-CON!",,콘서트,https://ticketimage.interpark.com/TicketImage/...
29,2025-07-23 12:00:00,4621,일반예매,더 로즈(The Rose) Once Upon A WRLD Tour in Seoul,25009724,콘서트,https://ticketimage.interpark.com/Play/image/l...
43,2025-07-28 20:00:00,3267,일반예매,2025 TEEN TOP WE GONNA ROCK IT DROP IT TOP IT ...,,콘서트,https://ticketimage.interpark.com/TicketImage/...
13,2025-07-21 20:00:00,2452,일반예매,2025 RESCENE 1st FAN-CON ［Project 326］,25010172,콘서트,https://ticketimage.interpark.com/Play/image/l...
28,2025-07-22 20:00:00,1706,일반예매,2025 유채훈 크로스오버 콘서트 〈IL MONDO〉,,콘서트,https://ticketimage.interpark.com/TicketImage/...
30,2025-07-23 16:00:00,1468,일반예매,PEAKBOX 2025 : 사랑,25010465,콘서트,https://ticketimage.interpark.com/Play/image/l...


In [2]:


import os
import json
import requests
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from openai import OpenAI
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import random
from datetime import datetime

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class InterparkTicketCrawler:
    def __init__(self, creds='google.json', sheet_name='감사한 티켓팅 신청서'):
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(creds, scope)
        self.sheet = gspread.authorize(creds).open(sheet_name).worksheet('Hot')

        # 캐시 파일 경로
        self.artist_cache_path = Path('artist_cache.json')
        self.hashtag_cache_path = Path('hashtag_cache.json')
        self.tweet_cache_path = Path('tweet_cache.json')

        # 캐시 로딩
        self.artist_cache = self.load_cache(self.artist_cache_path)
        self.hashtag_cache = self.load_cache(self.hashtag_cache_path)
        self.tweet_cache = self.load_cache(self.tweet_cache_path)
        
    def load_cache(self, path: Path) -> dict:
        if path.exists():
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                pass
        return {}

    def save_cache(self, cache: dict, path: Path):
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(cache, f, ensure_ascii=False, indent=2)

    def fetch_data(self):
        url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"
        params = {"goodsGenre": "ALL", "goodsRegion": "ALL", "offset": 0, "pageSize": 400, "sorting": "OPEN_ASC"}
        headers = {
            "user-agent": "Mozilla/5.0",
            "referer": "https://tickets.interpark.com/contents/notice"
        }
        r = requests.get(url, params=params, headers=headers)
        r.raise_for_status()
        return r.json()
# 뮤지컬, 연극 500 , 클래식/오페라 400, 콘서트 600
    def filter_hot(self, data):
        hot = []
        for d in data:
            if d.get('goodsGenreStr') == '콘서트' and d.get('viewCount', 0) <= 600:
                continue
            if d.get('goodsGenreStr') == '뮤지컬' and d.get('viewCount', 0) <= 500:
                continue
            if d.get('goodsGenreStr') == '연극' and d.get('viewCount', 0) <= 500:
                continue
            if d.get('goodsGenreStr') == '클래식/오페라' and d.get('viewCount', 0) <= 400:
                continue
            
            hot.append({
                '오픈시간': d.get('openDateStr', ''),
                '조회수': d.get('viewCount', 0),
                '예매타입': d.get('openTypeStr', ''),
                '제목': d.get('title', ''),
                '예매코드': d.get('goodsCode', ''),
                '장르': d.get('goodsGenreStr', ''),
                'Image': d.get('posterImageUrl', '')
            })
        return hot

    def extract_artist(self, title: str) -> str:
        if title in self.artist_cache:
            return self.artist_cache[title]

        prompt = f"""
아래는 콘서트 제목이야. 여기서 가수명이나 그룹명만 간단히 추출해줘. 뮤지컬일 경우 뮤지컬 제목만 추출해줘.**영문일 경우 한글도 같이 작성해야되고, 약어가 있으면 풀네임이랑 약어도 같이 작성해야해**
예시: 악동뮤지션 (악뮤, AKMU)
제목: {title}
가수명 or 뮤지컬 제목:"""

        try:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
            )
            artist = res.choices[0].message.content.strip().strip('"')
            self.artist_cache[title] = artist
            return artist
        except Exception as e:
            print(f"❌ OpenAI 오류 (가수명): {e}")
            return "불명"

    def generate_hashtags(self, title: str, artist: str, genre: str) -> str:
        key = f"{title}"
        if key in self.hashtag_cache:
            return self.hashtag_cache[key]

        prompt = f"""
콘서트 제목: {title}
가수 또는 뮤지컬 제목: {artist}
장르: {genre}

위 콘서트를 대리티켓팅 목적으로 트위터에 해시태그 10개를 한국어로 작성해줘.
형식: #블랙핑크콘서트 #블랙핑크 #BLACKPINK #블핑댈티 #대리티켓팅
조건: '#' 포함하고 띄어쓰기 없이, 한 줄로 콤마 없이 출력해줘. 키워드당 9자 이하여야해
"""

        try:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
            )
            hashtags = res.choices[0].message.content.strip()
            self.hashtag_cache[key] = hashtags
            return hashtags
        except Exception as e:
            print(f"❌ OpenAI 오류 (해시태그): {e}")
            return "#대리티켓팅"

    def add_ai_columns(self, df):
        print("🤖 가수명 + 해시태그 생성 중...")
        artists = []
        hashtags = []

        for _, row in tqdm(df.iterrows(), total=len(df)):
            title = row['제목']
            genre = row['장르']

            artist = self.extract_artist(title)
            hashtag = self.generate_hashtags(title, artist, genre)

            artists.append(artist)
            hashtags.append(hashtag)

        df['가수명'] = artists
        df['해시태그'] = hashtags

        self.save_cache(self.artist_cache, self.artist_cache_path)
        self.save_cache(self.hashtag_cache, self.hashtag_cache_path)

        return df
    
    def add_twitter_columns(self, df):
        print("🤖 트위터 문구 생성 중...")
        with open('tweet_templates.json', 'r', encoding='utf-8') as f:
            templates = json.load(f)
        
        tweet_contents = []
        for _, row in tqdm(df.iterrows(), total=len(df)):
            

            template = {"content": "{title}\n\n🚨 {singer} 대리티켓팅(댈티)\n\n수고비 제일 저렴\n경력 매우 많음\n\n상담 링크: https://open.kakao.com/o/sAJ8m2Ah\n\n{hash_tag}"}
            
            # if row['조회수'] > 10000:
            #     template = random.choice(templates)
            # else:
            #     template = {"content": "{title}\n\n🚨 {singer} 대리티켓팅(댈티)\n\n수고비 제일 저렴\n경력 매우 많음\n\n상담 링크: https://open.kakao.com/o/sAJ8m2Ah\n\n{hash_tag} #평생한번 #놓치면후회 #앞열보장"}
            
            # 시간 치환
            
            title = row['제목']
            singer = row['가수명']
            
            # 오픈시간이 문자열인 경우 datetime으로 변환
            open_time_raw = row['오픈시간']
            if isinstance(open_time_raw, str):
                # 문자열을 datetime으로 변환
                open_time_dt = datetime.strptime(open_time_raw, '%Y-%m-%d %H:%M:%S')
                open_time = open_time_dt.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            else:
                # 이미 datetime 객체인 경우
                open_time = open_time_raw.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            
            hash_tag = row['해시태그']
            content = template['content'].replace("{open_time}", open_time).replace("{title}", title).replace("{singer}", singer).replace("{hash_tag}", hash_tag)
            tweet_contents.append(content)

        df['트위터'] = tweet_contents
        self.save_cache(self.tweet_cache, self.tweet_cache_path)
        return df
        
    def bunjang_columns(self, df):
        print("🤖 번장 문구 생성 중...")
        
        bunjang_contents = []
        for _, row in tqdm(df.iterrows(), total=len(df)):
            

            template = {"content": "{title}\n\n🚨 {singer} 대리티켓팅(댈티)\n\n수고비 제일 저렴\n경력 매우 많음\n\n가격: 번개톡 상담\n\n{hash_tag}"}
            
            
            title = row['제목']
            singer = row['가수명']
            
            # 오픈시간이 문자열인 경우 datetime으로 변환
            open_time_raw = row['오픈시간']
            if isinstance(open_time_raw, str):
                # 문자열을 datetime으로 변환
                open_time_dt = datetime.strptime(open_time_raw, '%Y-%m-%d %H:%M:%S')
                open_time = open_time_dt.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            else:
                # 이미 datetime 객체인 경우
                open_time = open_time_raw.strftime('%m월 %d일 %p %I시').replace('AM', '오전').replace('PM', '오후').replace('0', '')
            
            hash_tag = row['해시태그']
            content = template['content'].replace("{open_time}", open_time).replace("{title}", title).replace("{singer}", singer).replace("{hash_tag}", hash_tag)
            bunjang_contents.append(content)

        df['번장'] = bunjang_contents
        
        return df

    def update_sheet(self, df):
        self.sheet.clear()
        if df.empty:
            print("📭 HOT 티켓 없음")
            return
        self.sheet.append_row(list(df.columns))
        for row in df.values.tolist():
            self.sheet.append_row(row)
        print(f"✅ {len(df)}개 티켓 업로드 완료")

    def run(self):
        raw = self.fetch_data()
        hot = self.filter_hot(raw)
        df = pd.DataFrame(hot)
        df = df[df['오픈시간'].notna() & (df['오픈시간'] != '')]
        if df.empty:
            return df
        df = df.sort_values(by='오픈시간')
        df = self.add_ai_columns(df)
        df = self.add_twitter_columns(df)
        df = self.bunjang_columns(df)
        self.update_sheet(df)
        return df

if __name__ == "__main__":
    df = InterparkTicketCrawler().run()
    if not df.empty:
        print("\n📋 HOT 티켓 요약:")
        
        # print(df[['오픈시간', '제목', '가수명', '해시태그', '번장', '트위터']].to_string(index=False))

🤖 가수명 + 해시태그 생성 중...


100%|██████████| 31/31 [00:05<00:00,  5.69it/s]


🤖 트위터 문구 생성 중...


100%|██████████| 31/31 [00:00<00:00, 31091.21it/s]


🤖 번장 문구 생성 중...


100%|██████████| 31/31 [00:00<?, ?it/s]


✅ 31개 티켓 업로드 완료

📋 HOT 티켓 요약:


In [20]:
import os
import requests
import json
from pathlib import Path
from time import sleep
import random
from tqdm import tqdm

import os
import requests
import json
from pathlib import Path
from time import sleep
import random
from tqdm import tqdm

class PostBunjang:
    def __init__(self, auth_token=None):
        self.auth_token = "53a119a23abe4baa83d75e604dbc2a2d"
        self.location = {
            "address": "서울특별시 서초구 서초4동",
            "lat": 37.5025863,
            "lon": 127.022219,
            "dongId": 648
        }
        os.makedirs("image", exist_ok=True)

    def _download_image(self, url):
        path = f"image/{url.split('/')[-1]}"
        if os.path.exists(path):
            print(f"📁 이미지 이미 존재: {path}")
            return path
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(path, "wb") as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
            print(f"✅ 이미지 다운로드 완료: {path}")
            return path
        print(f"❌ 이미지 다운로드 실패: {url}")
        return None

    def register_bunjang_product(self, image_path, name, description, keywords, price):
        # 1단계: 이미지 업로드
        upload_url = 'https://media-center.bunjang.co.kr/upload/79373298/product'
        upload_headers = {
            'referer': 'https://m.bunjang.co.kr/',
            'user-agent': 'Mozilla/5.0',
            'origin': 'https://m.bunjang.co.kr',
            'accept': 'application/json, text/plain, */*'
        }

        if not Path(image_path).exists():
            print(f"❌ 이미지 파일 없음: {image_path}")
            return None

        with open(image_path, 'rb') as img_file:
            # ✅ 파일명은 항상 ASCII (latin-1 인코딩 문제 방지)
            files = {'file': ('upload.jpg', img_file, 'image/jpeg')}
            upload_res = requests.post(upload_url, headers=upload_headers, files=files)

        if upload_res.status_code != 200:
            print("❌ 이미지 업로드 실패:", upload_res.text)
            return None

        image_id = upload_res.json().get('image_id')
        print("✅ 이미지 업로드 성공:", image_id)

        # 2단계: 상품 등록
        product_url = 'https://api.bunjang.co.kr/api/pms/v2/products'
        product_headers = {
            'content-type': 'application/json',
            'x-bun-auth-token': self.auth_token,
            'user-agent': 'Mozilla/5.0',
            'origin': 'https://m.bunjang.co.kr',
            'referer': 'https://m.bunjang.co.kr/',
            'accept': 'application/json, text/plain, */*'
        }

        # 해시태그 문자열이면 리스트로 변환
        if isinstance(keywords, str):
            keywords = [k.strip() for k in keywords.split('#') if k.strip()]

        product_data = {
            "categoryId": "900210001",
            "common": {
                "description": description,
                "keywords": keywords,
                "name": name,
                "condition": "UNDEFINED",
                "priceOfferEnabled": True
            },
            "option": [],
            "location": {"geo": self.location},
            "transaction": {
                "quantity": 1,
                "price": price,
                "trade": {
                    "freeShipping": True,
                    "isDefaultShippingFee": False,
                    "inPerson": True
                }
            },
            "media": [{"imageId": image_id}],
            "naverShoppingData": {"isEnabled": False}
        }

        res = requests.post(product_url, headers=product_headers, json=product_data)

        if res.status_code == 200:
            pid = res.json().get("data", {}).get("pid", "N/A")
            print("✅ 상품 등록 성공! 🆔", pid)
            return pid
        else:
            print(f"❌ 상품 등록 실패 ({res.status_code}): {res.text}")
            return None

    def post(self, image_url, title, text, hash_tag, price):
        path = self._download_image(image_url)
        if not path:
            return
        pid = self.register_bunjang_product(
            image_path=path,
            name=title,
            description=text,
            keywords=hash_tag,
            price=price
        )
        if pid:
            print(f"🔗 번장 링크: https://m.bunjang.co.kr/products/{pid}")


# ✅ 예시 실행 (df는 미리 정의된 pandas DataFrame이어야 함)
for _, row in tqdm(df.iterrows(), total=len(df)):
    title = row['가수명'] + " 대리티켓팅(댈티)"
    text = row['번장']
    image_url = row['Image']
    tag_str = row['해시태그']
    hash_tag = [tag.strip().lstrip('#')[:8] for tag in tag_str.split()][:5]

    price = 9999

    PostBunjang().post(image_url, title, text, hash_tag, price)
    print(f"🔄 {title} 번장 게시 완료")

    sleep_time = random.randint(60, 90)
    for remaining in range(sleep_time, 0, -1):
        print(f"\r⏰ 다음 게시까지 {remaining}초 남음...", end="", flush=True)
        sleep(1)
    print(f"\n⏳ {sleep_time}초 대기 완료")

  0%|          | 0/29 [00:00<?, ?it/s]

📁 이미지 이미 존재: image/25010172_p.gif
✅ 이미지 업로드 성공: 1487367030
✅ 상품 등록 성공! 🆔 346260793
🔗 번장 링크: https://m.bunjang.co.kr/products/346260793
🔄 RESCENE 대리티켓팅(댈티) 번장 게시 완료
⏰ 다음 게시까지 1초 남음....

  3%|▎         | 1/29 [01:15<35:23, 75.85s/it]


⏳ 75초 대기 완료
✅ 이미지 다운로드 완료: image/25010481_p.gif
✅ 이미지 업로드 성공: 1487375405
✅ 상품 등록 성공! 🆔 346261048
🔗 번장 링크: https://m.bunjang.co.kr/products/346261048
🔄 태양의서커스 (Cirque du Soleil) 대리티켓팅(댈티) 번장 게시 완료
⏰ 다음 게시까지 1초 남음....

  7%|▋         | 2/29 [02:45<37:55, 84.26s/it]

...


⏳ 78초 대기 완료
✅ 이미지 다운로드 완료: image/20250716111436.jpg
✅ 이미지 업로드 성공: 1487400740
✅ 상품 등록 성공! 🆔 346267621
🔗 번장 링크: https://m.bunjang.co.kr/products/346267621
🔄 윤석철트리오, H ZETTRIO (에이치 제트 트리오) 대리티켓팅(댈티) 번장 게시 완료
⏰ 다음 게시까지 1초 남음....

 97%|█████████▋| 28/29 [34:23<01:12, 72.52s/it]


⏳ 67초 대기 완료
✅ 이미지 다운로드 완료: image/20250721111530.jpg
✅ 이미지 업로드 성공: 1487401562
✅ 상품 등록 성공! 🆔 346267858
🔗 번장 링크: https://m.bunjang.co.kr/products/346267858
🔄 세븐틴 (SEVENTEEN) 대리티켓팅(댈티) 번장 게시 완료
⏰ 다음 게시까지 1초 남음....

100%|██████████| 29/29 [35:49<00:00, 74.14s/it]


⏳ 85초 대기 완료


In [21]:
import os
import tweepy
import requests
from dotenv import load_dotenv
import time
import random
from tqdm import tqdm

load_dotenv()

class PostTweet:
    def __init__(self):
        auth = tweepy.OAuthHandler(os.getenv('TWITTER_API_KEY'), os.getenv('TWITTER_API_SECRET'))
        auth.set_access_token(os.getenv('TWITTER_ACCESS_TOKEN'), os.getenv('TWITTER_ACCESS_TOKEN_SECRET'))
        self.api = tweepy.API(auth)
        self.client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )

    def _download_image(self, url):
        path = f"image/{url.split('/')[-1]}"
        if os.path.exists(path):
            print(f"이미지 {path} 이미 다운로드됨, 스킵합니다.")
            return path
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(path, "wb") as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
            return path
        return None

    def post(self, text, image_url=None):
        media_ids = []
        if image_url:
            path = self._download_image(image_url)
            if path:
                media = self.api.media_upload(path)
                media_ids.append(media.media_id)

        tweet = self.client.create_tweet(text=text, media_ids=media_ids if media_ids else None)
        print(f"https://twitter.com/gamsahanticket/status/{tweet.data['id']}")


for _, row in tqdm(df.iterrows(), total=len(df)):
    title = row['제목']
    text = row['트위터']
    image_url = row['Image']
    PostTweet().post(text, image_url)
    print(f"🔄 {title} 트윗 게시 완료")
    sleep_time = random.randint(60, 90)
    
    # 실시간 카운트다운
    for remaining in range(sleep_time, 0, -1):
        print(f"\r⏰ 다음 트윗까지 {remaining}초 남음...", end="", flush=True)
        time.sleep(1)
    
    print(f"\n🔄 {sleep_time}초 대기 완료, 다음 트윗 게시")


  0%|          | 0/29 [00:00<?, ?it/s]

이미지 image/25010172_p.gif 이미 다운로드됨, 스킵합니다.
https://twitter.com/gamsahanticket/status/1947245951825113219
🔄 2025 RESCENE 1st FAN-CON ［Project 326］ 트윗 게시 완료
⏰ 다음 트윗까지 1초 남음....

  3%|▎         | 1/29 [01:31<42:44, 91.58s/it]


🔄 90초 대기 완료, 다음 트윗 게시
이미지 image/25010481_p.gif 이미 다운로드됨, 스킵합니다.
https://twitter.com/gamsahanticket/status/1947246336610635819
🔄 ［NOL 스페셜 스테이지］ 태양의서커스 〈쿠자〉 - 부산 트윗 게시 완료
⏰ 다음 트윗까지 1초 남음....

  7%|▋         | 2/29 [02:46<36:44, 81.66s/it]

...


🔄 88초 대기 완료, 다음 트윗 게시
이미지 image/25006050_p.gif 이미 다운로드됨, 스킵합니다.
https://twitter.com/gamsahanticket/status/1947251100840820940
🔄 뮤지컬 〈프리다〉  트윗 게시 완료
⏰ 다음 트윗까지 1초 남음....

 59%|█████▊    | 17/29 [21:48<16:02, 80.17s/it]


🔄 79초 대기 완료, 다음 트윗 게시
이미지 image/25005672_p.gif 이미 다운로드됨, 스킵합니다.


 59%|█████▊    | 17/29 [21:48<15:23, 77.00s/it]


TooManyRequests: 429 Too Many Requests
Too Many Requests

In [ ]:
import os
import tweepy
import requests
from dotenv import load_dotenv
import time
import random
from tqdm import tqdm
from requests_oauthlib import OAuth1

load_dotenv()

class PostTweet:
    def __init__(self):
        # OAuth 1.0 인증
        self.auth = tweepy.OAuthHandler(os.getenv('TWITTER_API_KEY'), os.getenv('TWITTER_API_SECRET'))
        self.auth.set_access_token(os.getenv('TWITTER_ACCESS_TOKEN'), os.getenv('TWITTER_ACCESS_TOKEN_SECRET'))
        self.api = tweepy.API(self.auth)

        # OAuth 2.0 클라이언트
        self.client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )

    def _download_image(self, url):
        path = f"image/{url.split('/')[-1]}"
        if os.path.exists(path):
            print(f"📁 이미지 {path} 이미 존재. 스킵함.")
            return path
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(path, "wb") as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
            return path
        print("❌ 이미지 다운로드 실패")
        return None

    def _safe_media_upload(self, path):
        try:
            media = self.api.media_upload(path)
            return media.media_id
        except tweepy.TooManyRequests as e:
            reset_ts = int(e.response.headers.get("x-rate-limit-reset", time.time() + 900))
            wait_time = int(reset_ts - time.time()) + 5
            print(f"💣 이미지 업로드 제한! {wait_time}초 대기")
            time.sleep(wait_time)
            return None
        except Exception as e:
            print(f"❌ 이미지 업로드 에러: {e}")
            return None

    def post(self, text, image_url=None):
        media_ids = []
        if image_url:
            path = self._download_image(image_url)
            if path:
                media_id = self._safe_media_upload(path)
                if media_id:
                    media_ids.append(media_id)

        try:
            tweet = self.client.create_tweet(text=text, media_ids=media_ids if media_ids else None)
            tweet_url = f"https://twitter.com/gamsahanticket/status/{tweet.data['id']}"
            print(f"✅ 트윗 성공: {tweet_url}")
        except tweepy.TooManyRequests as e:
            reset_ts = int(e.response.headers.get("x-rate-limit-reset", time.time() + 900))
            print(e.response.headers)
            wait_time = int(reset_ts - time.time()) + 5
            print(f"🚫 텍스트 트윗 Rate Limit! {wait_time}초 대기")
            time.sleep(wait_time)
        except Exception as e:
            print(f"❌ 트윗 에러: {e}")

import pandas as pd
import time
import random
from tqdm import tqdm

# 예시 DataFrame

for _, row in tqdm(df.iterrows(), total=len(df)):
    title = row['제목']
    text = row['트위터']
    image_url = row['Image']

    PostTweet().post(text, image_url)
    print(f"🔄 {title} 트윗 완료")

    sleep_time = random.randint(60, 90)
    for remaining in range(sleep_time, 0, -1):
        print(f"\r⏰ 다음 트윗까지 {remaining}초 남음...", end="", flush=True)
        time.sleep(1)
    print(f"\n⏱ {sleep_time}초 대기 완료")


  0%|          | 0/31 [00:00<?, ?it/s]

{'Date': 'Mon, 21 Jul 2025 11:31:35 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '94', 'Connection': 'keep-alive', 'perf': '7402827104', 'Set-Cookie': 'guest_id_marketing=v1%3A175309749535876950; Max-Age=63072000; Expires=Wed, 21 Jul 2027 11:31:35 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, guest_id_ads=v1%3A175309749535876950; Max-Age=63072000; Expires=Wed, 21 Jul 2027 11:31:35 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, personalization_id="v1_QjaQ0hp7CyfocfAKZe4NiA=="; Max-Age=63072000; Expires=Wed, 21 Jul 2027 11:31:35 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, guest_id=v1%3A175309749535876950; Max-Age=63072000; Expires=Wed, 21 Jul 2027 11:31:35 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, __cf_bm=65LSiHHFoT1dRIvO1BI1WActQaJyReC4p.j0QihrbHI-1753097495-1.0.1.1-dIB7uZxM_qaG7FGoP3nXWOf9nKxxJplwMkMMm0ZN4iMdGE6_8laeG6FfmhGDpRz5oh6Yfc71L1QdvgBhiPGWd0FP62AKfUhQflEPrxu0FZg; path=/; expires=Mon, 21-Jul-25 12:01

In [ ]:
{'Date': 'Mon, 21 Jul 2025 11:31:35 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '94', 'Connection': 'keep-alive', 'perf': '7402827104', 'Set-Cookie': 'guest_id_marketing=v1%3A175309749535876950; Max-Age=63072000; Expires=Wed, 21 Jul 2027 11:31:35 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, guest_id_ads=v1%3A175309749535876950; Max-Age=63072000; Expires=Wed, 21 Jul 2027 11:31:35 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, personalization_id="v1_QjaQ0hp7CyfocfAKZe4NiA=="; Max-Age=63072000; Expires=Wed, 21 Jul 2027 11:31:35 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, guest_id=v1%3A175309749535876950; Max-Age=63072000; Expires=Wed, 21 Jul 2027 11:31:35 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, __cf_bm=65LSiHHFoT1dRIvO1BI1WActQaJyReC4p.j0QihrbHI-1753097495-1.0.1.1-dIB7uZxM_qaG7FGoP3nXWOf9nKxxJplwMkMMm0ZN4iMdGE6_8laeG6FfmhGDpRz5oh6Yfc71L1QdvgBhiPGWd0FP62AKfUhQflEPrxu0FZg; path=/; expires=Mon, 21-Jul-25 12:01:35 GMT; domain=.twitter.com; HttpOnly; Secure; SameSite=None', 'api-version': '2.144', 'Cache-Control': 'no-cache, no-store, max-age=0', 'x-access-level': 'read-write', 'x-frame-options': 'SAMEORIGIN', 'Content-Encoding': 'gzip', 'x-transaction-id': '90fb7ee6c917f42f', 'x-xss-protection': '0', 'x-rate-limit-limit': '1080000', 'x-rate-limit-reset': '1753098288', 'content-disposition': 'attachment; filename=json.json', 'x-content-type-options': 'nosniff', 'x-rate-limit-remaining': '1079998', 'x-app-limit-24hour-limit': '17', 'x-app-limit-24hour-reset': '1753180987', 'x-user-limit-24hour-limit': '17', 'x-user-limit-24hour-reset': '1753180987', 'x-app-limit-24hour-remaining': '0', 'x-user-limit-24hour-remaining': '0', 'strict-transport-security': 'max-age=631138519; includeSubdomains', 'x-response-time': '14', 'x-connection-hash': '8ca9699180aa87c99bfa24e3a58493237d02af8010426cd921025db14411db1a', 'vary': 'accept-encoding', 'cf-cache-status': 'DYNAMIC', 'Server': 'cloudflare tsa_p', 'CF-RAY': '962a5f717a94ea10-ICN'}
